# Solving for the Chebyshev-Based Spectral Derivative at Domain Endpoints

There is a major problem in the expanded, "splintered" expressions for $\frac{d^\nu}{dx^\nu} y(\theta)$: Factors in $x$ multiplying each $y{(\nu)} = \frac{d^\nu}{d\theta^\nu}$ have multiples of $\sqrt{1-x^2}$ in their denominators, making them undefined for $x = \pm 1$. We can solve for the endpoints using a L'Hôpital's rule, but the answers are difficult to derive, especially for higher derivatives.

I opened [Issue #1](https://github.com/pavelkomarov/spectral-derivatives/issues/1) to address the ugliness of this with a programmatic solution. An algorithm to derive the endpoint expressions turns out to be pretty challenging, not so easy to turn into compact code, and unfortunately relies on symbolic differentiation and simplification. As such, it has not been put in to the core library code. However, in case anyone should need to take a derivative beyond the $4^{th}$, this notebook implements a process to find the right expressions.

In [1]:
import sympy as sp
from collections import defaultdict, deque
from numpy.polynomial import Polynomial as poly

In [2]:
order = 4 # The order of derivative you want to work with

## Using $\theta$ Instead of $x$

$$ \frac{d}{dx} y(\theta) = \frac{d}{d\theta}y(\theta) \cdot \frac{d\theta}{dx}$$

And then subsequent derivatives entail a product rule, which is how terms quickly proliferate. These derivatives are pretty ugly, because $\frac{d\theta}{dx} = \frac{d}{dx}\cos^{-1}(x) = \frac{-1}{\sqrt{1-x^2}}$. However, due to the relationship $x = \cos(\theta)$, $\frac{-1}{\sqrt{1-x^2}} = \frac{-1}{\sin(\theta)}$. To avoid taking the limit of a quantity with multiple related variables, it's somewhat cleaner to put everything in terms of $\theta$ and then apply L'Hôpital's rule by taking derivatives w.r.t. $\theta$ rather than $x$. The two are equivalent; we end up with powers of $\sin(\theta)$ in the denominator, which is also stubbornly 0 at the endpoints ($\theta = 0$ and $\pi$).

## Finding the Denominator

If we carry through with L'Hôpital's rule in variable $\theta$, we see that after each differentation a $\sin(\theta)$ cancels with all the terms in the numerator, just like how when we do it in $x$ a $\sqrt{1-x^2}$ cancels, and we can thereby, over the course of $\nu$ applications of the rule, eliminate the troublesome $\sin(\theta)$s in the denominator.

The denominator starts out $\sin^{2\nu-1}(\theta)$ and gets changed simply as in the following loop:

In [3]:
th = sp.symbols('th')
denom = sp.sin(th)**(2*order - 1)
for i in range(order-1):
	denom = sp.diff(denom, th)
	denom = sp.cancel(denom/sp.sin(th))
denom = sp.diff(denom, th).trigsimp()

We can now evaluate at the endpoints and get nonzero values. For even $\nu$ the two endpoints have the same denominator; for odd $\nu$ they are $\pm$ each other.

In [4]:
D_0 = denom.subs(th, 0)
D_pi = denom.subs(th, sp.pi)
print(D_0, D_pi)

105 105


## Finding the Numerator Terms

We can begin with the pyramid of terms built up for $\frac{d^\nu}{dx^\nu} y(\theta)$ as evaluated in the package.

In [5]:
# Calculate the polynomials in x necessary for transforming back to the Chebyshev domain
numers = deque([poly([-1])]) # just -1 to start, at order 1
denom = poly([1, 0, -1]) # 1 - x^2
for nu in range(2, order + 1): # initialization takes care of order 1, so iterate from order 2
	q = 0
	for mu in range(1, nu): # Terms come from the previous derivative, so there are nu - 1 of them here.
		p = numers.popleft() # c = nu - mu/2
		numers.append(denom * p.deriv() + (nu - mu/2 - 1) * poly([0, 2]) * p - q)
		q = p
	numers.append(-q)

These are functions in $x$, but we can substitute to easily make them functions of $\cos(\theta)$, and then we can multipy by appropriate powers of $\sin(\theta)$ to put them all in the numerator of a single fraction.

In [6]:
exprs = deque()
for mu,p in enumerate(numers):
	expr = 0
	for r,c in enumerate(p.coef):
		expr += c * sp.cos(th)**r
	expr *= sp.sin(th)**mu
	exprs.append(expr)

In [7]:
print(exprs)

deque([-6.0*cos(th)**3 - 9.0*cos(th), (11.0*cos(th)**2 + 4.0)*sin(th), -6.0*sin(th)**2*cos(th), 1.0*sin(th)**3])


Now we note that these are multiplying subsequent orders of $y^{(\mu)}(\theta)$, so the full numerator looks like:

$$a(\theta)y'(\theta) + b(\theta)y''(\theta) + ... z(\theta)y^{(\nu)}(\theta)$$

This situation will mean product rules as we take derivatives. Mildly gross, but like the pyramid scheme from before, each $y^{(\mu)}(\theta)$ depends only on two terms from the previous expression, so we can evaluate subsequent numerators actually pretty easily.

For example:

$$ay' + by'' + cy''' \rightarrow a'y' + (a + b')y'' + (b + c')y''' + cy^{IV}$$

We can line these up to see an obvious relationship of each new term to the one above and the one up-left-diagonal:

$$\begin{array}{c c c c}
a & b & c \\
a' & a+b' & b+c' & c\\
\end{array}$$

Then before the next iteration of this process we divide by a $\sin(\theta)$ to cancel one from the denominator.

We need to do this $\nu$ times to get all the way up to the $(2\nu)^{th}$ derivative before the denominator stops being 0. However, a beautiful thing happens to the numerator at the second-to-last of these additional iterations: All the terms involving even derivatives of $y(\theta)$ cancel out, and all the terms involving odd derivatives are multiplied by only *constant* factors. It's like a Law of Nature, and I'm not sure exactly why. Then the last derivative entails no product rule, only shifts the constant coefficients of the odd derivatives to the next-highest even derivatives. So we can really stop the iteration one short.

In [8]:
for nu in range(order + 1, 2*order): # iterate order-1 more times to reach the constants
	q = 0
	for mu in range(1, nu): # Terms come from the previous derivative, so there are nu - 1 of them here.
		p = exprs.popleft()
		term = (q + sp.diff(p, th)).simplify()
		term = sp.cancel(term/sp.sin(th)).trigsimp()
		exprs.append(term)
		q = p
	exprs.append(sp.cancel(q/sp.sin(th)).trigsimp())
	print(nu, exprs)

5 deque([18.0*cos(th)**2 + 9.0, -13.5*sin(2*th), 7.0*sin(th)**2 + 3.0, -1.5*sin(2*th), 1.0*sin(th)**2])
6 deque([-36.0*cos(th), 36.0*sin(th), -13.0*cos(th), 13.0*sin(th), -1.0*cos(th), 1.0*sin(th)])
7 deque([36.0000000000000, 0, 49.0000000000000, 0, 14.0000000000000, 0, 1.00000000000000])


In [9]:
C = [round(exprs[i]) for i in range(0, 2*order-1, 2)] # constants
print(C)

[36, 49, 14, 1]


We can pick out that $y''(\theta)$ needs to be multiplied by the first value, $y^{IV}(\theta)$ by the second, and so on.

## Putting it All Together

If we use the DCT$^{-1}$ to reconstruct the first and last points, the answer will be:

$$
\begin{align} \frac{1}{D_0 M} \Big((... - C_3 N^6 + C_2 N^4 - C_1 N^2) Y_N + 2 \sum_{k=1}^{N-1} (... - C_3 k^6 + C_2 k^4 - C_1 k^2) Y_k \Big) & \text{ at } x=1, \theta=0 \\ \frac{1}{D_\pi M} \Big((... - C_3 N^6 + C_2 N^4 - C_1 N^2)(-1)^N Y_N + 2 \sum_{k=1}^{N-1} (... - C_3 k^6 + C_2 k^4 - C_1 k^2) (-1)^k Y_k \Big) & \text{ at } x=-1, \theta=\pi \end{align}
$$

Where the alternating plus and minus come from the fact the $2^{nd}$ derivative contains `-cos`, the $4^{th}$ `cos`, the $6^{th}$ `-cos` again, and so on.

Let's make a code string that implements that, for the `order` given.

In [10]:
Ns = "".join((" + " if i % 2 else " - ") + f"{C[i]}" + (f"*N**{i*2}" if i > 0 else "") for i in range(len(C)-1, -1, -1))
ks = "".join((" + " if i % 2 else " - ") + f"{C[i]}" + f"*k**{(i+1)*2}" for i in range(len(C)-1, -1, -1))
d = {"- -":"+ ", "+ -":"- ", " + 1*":"", " 1*":" "}
for k,v in d.items(): Ns = Ns.replace(k, v)
for k,v in d.items(): ks = ks.replace(k, v)

print(f"dy_n[first] = np.sum(({ks})[s] * Y_k[middle], axis=axis)/({D_0}*N) + N*({Ns})/{2*D_0} * Y_k[last]")
print(f"dy_n[last] = np.sum((({ks})*np.power(-1, k))[s] * Y_k[middle], axis=axis)/({D_pi}*N) + (N*({Ns})*(-1)**N)/{2*D_pi} * Y_k[last]")

dy_n[first] = np.sum((k**8 - 14*k**6 + 49*k**4 - 36*k**2)[s] * Y_k[middle], axis=axis)/(105*N) + N*(N**6 - 14*N**4 + 49*N**2 - 36)/210 * Y_k[last]
dy_n[last] = np.sum(((k**8 - 14*k**6 + 49*k**4 - 36*k**2)*np.power(-1, k))[s] * Y_k[middle], axis=axis)/(105*N) + (N*(N**6 - 14*N**4 + 49*N**2 - 36)*(-1)**N)/210 * Y_k[last]
